In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
from datetime import datetime, timedelta
from fake_useragent import UserAgent
import fake_useragent
# df = pd.read_excel("imdb-dataset.xlsx")
df = pd.read_excel("full_list_of_titles.xlsx")
df.head()

,name,language,plot,story,actor1,actor2,actor3,director1,director2,director3,...,executive_producer3,line_producer1,line_producer2,line_producer3,co_producer1,co_producer2,co_producer3,associate_producer1,associate_producer2,associate_producer3
0,House of Cards,English,A Congressman works with his equally conniving...,Majority House Whip Francis Underwood takes yo...,Robin Wright,Michael Kelly,Kevin Spacey,James Foley,Robin Wright,John David Coles,...,Robert Zotnowski,NaN,NaN,NaN,Jay Carson,Boris Malden,Peter Mavromates,Elizabeth A. Lyons,Jesse Rose-Smith,NaN
1,Hemlock Grove,English,Secrets are just part of daily life in the sma...,Secrets are just part of daily life in the sma...,Famke Janssen,Bill Skarsgard,Landon Liboiron,David Straiton,Deran Sarafian,Russell Lee Fine,...,Elisa Todd Ellis,Lena Cordina,NaN,NaN,Tim King,Rafe Judkins,Lauren LeFranc,Gregg Simon,Christina Perez,Megan Driscoll
2,Orange Is the New Black,English,Convicted of a decade old crime of transportin...,Piper Chapman is a public relations executive ...,Taylor Schilling,Kate Mulgrew,Uzo Aduba,Andrew McCarthy,Phil Abraham,Michael Trim,...,Jenji Kohan,NaN,NaN,NaN,Arturo Guzman,David Price,Aimee Roth,NaN,NaN,NaN
3,Marco Polo,Persian\n|\nUighur\n|\nItalian\n|\nMongolian\n...,"In a world replete with greed, betrayal, sexua...","In a world replete with greed, betrayal, sexua...",Lorenzo Richelmy,Benedict Wong,Joan Chen,Daniel Minahan,David Petrarca,Alik Sakharov,...,Dave Erickson,Tim Coddington,Akhat Ibraev,Guy Tannahill,Robyn-Alain Feldman,NaN,NaN,Sean Guest,Alexandra Canosa,Russ Hammonds
4,Bloodline,English,A family finds themselves forced to face their...,Intriguing story of a well off family in the F...,Kyle Chandler,Linda Cardellini,Norbert Leo Butz,Mikael Håfström,Todd A. Kessler,Ed Bianchi,...,Mark A. Baker,NaN,NaN,NaN,Greer Yeaton,Jeff Shakoor,NaN,Deen Dioria,Michael Carroll,NaN


In [2]:
df.columns

Index(['name', 'language', 'plot', 'story', 'actor1', 'actor2', 'actor3',
       'director1', 'director2', 'director3', 'writer1', 'writer2', 'writer3',
       'executive_producer1', 'executive_producer2', 'executive_producer3',
       'line_producer1', 'line_producer2', 'line_producer3', 'co_producer1',
       'co_producer2', 'co_producer3', 'associate_producer1',
       'associate_producer2', 'associate_producer3'],
      dtype='object')

In [3]:
# Extracting release info based on country and url where country = "India" or "USA"
def get_release_date(url, country="India"):
    res = requests.get(url)
    html_page = res.content
    soup = BeautifulSoup(html_page, "html.parser")
    content = soup.find_all("tr", {"class": "ipl-zebra-list__item release-date-item"})

    for line in content:
        if country in line.text:
            lines = (str(line.text).strip()).split("\n")
            lines = [line[0:18].strip() for line in lines if len(line[0:18].strip())>8 and line != country]
            if len(lines) > 0 and len(lines[0].split()) == 3:
                release_date = datetime.strptime(lines[0], "%d %B %Y")
                week_after_release = release_date + timedelta(days=7)
                return [str(datetime.date(release_date)), str(datetime.date(week_after_release))]
    return ["N/A", "N/A"]


In [159]:
# India
# Searching on IMDb search bar and reaching inside a item's page
url = "https://www.imdb.com/find?q="
release = []
week_after_release = []
for indx in df["name"].index:
    res = requests.get(url+df["name"][indx])
    html_page = res.content
    soup = BeautifulSoup(html_page, "html.parser")
    content = soup.find("div", {"class": "findSection"})
    # xpath = //div[@class='txt-block']//h4[contains(text(), 'Release Date')]//following-sibling::span
    
    show_url = "https://www.imdb.com" + str(content.table.a['href']) + str("releaseinfo")
    sample = get_release_date(show_url, country="India")
    if len(sample) == 2:
        release.append(sample[0])
        week_after_release.append(sample[1])
    else:
        release.append("N/A")
        week_after_release.append("N/A")
names = df
se = pd.Series(release)
names['release date'] = se.values
se = pd.Series(week_after_release)
names['seven days after'] = se.values
names.to_csv("output-india-2.csv")

In [158]:
# USA
# Searching on IMDb search bar and reaching inside a item's page
url = "https://www.imdb.com/find?q="
release = []
week_after_release = []
for indx in df["name"].index:
    res = requests.get(url+df["name"][indx])
    html_page = res.content
    soup = BeautifulSoup(html_page, "html.parser")
    content = soup.find("div", {"class": "findSection"})
    # xpath = //div[@class='txt-block']//h4[contains(text(), 'Release Date')]//following-sibling::span
    
    show_url = "https://www.imdb.com" + str(content.table.a['href']) + str("releaseinfo")
    sample = get_release_date(show_url, country="USA")
    if len(sample) == 2:
        release.append(sample[0])
        week_after_release.append(sample[1])
    else:
        release.append("N/A")
        week_after_release.append("N/A")
names = df
se = pd.Series(release)
names['release date'] = se.values
se = pd.Series(week_after_release)
names['seven days after'] = se.values
names.to_csv("output-usa.csv")

In [144]:
df["name"][indx]

'Dolly Kitty and Those Twinkling Stars\xa0(2019)'

In [145]:
show_url

'https://www.imdb.com/title/tt9176296/releaseinfo'

In [160]:
df.head(10)

,name,language,plot,story,actor1,actor2,actor3,director1,director2,director3,...,line_producer2,line_producer3,co_producer1,co_producer2,co_producer3,associate_producer1,associate_producer2,associate_producer3,release date,seven days after
0,House of Cards,English,A Congressman works with his equally conniving...,Majority House Whip Francis Underwood takes yo...,Robin Wright,Michael Kelly,Kevin Spacey,James Foley,Robin Wright,John David Coles,...,NaN,NaN,Jay Carson,Boris Malden,Peter Mavromates,Elizabeth A. Lyons,Jesse Rose-Smith,NaN,2014-02-20,2014-02-27
1,Hemlock Grove,English,Secrets are just part of daily life in the sma...,Secrets are just part of daily life in the sma...,Famke Janssen,Bill Skarsgard,Landon Liboiron,David Straiton,Deran Sarafian,Russell Lee Fine,...,NaN,NaN,Tim King,Rafe Judkins,Lauren LeFranc,Gregg Simon,Christina Perez,Megan Driscoll,N/A,N/A
2,Orange Is the New Black,English,Convicted of a decade old crime of transportin...,Piper Chapman is a public relations executive ...,Taylor Schilling,Kate Mulgrew,Uzo Aduba,Andrew McCarthy,Phil Abraham,Michael Trim,...,NaN,NaN,Arturo Guzman,David Price,Aimee Roth,NaN,NaN,NaN,2017-10-04,2017-10-11
3,Marco Polo,Persian\n|\nUighur\n|\nItalian\n|\nMongolian\n...,"In a world replete with greed, betrayal, sexua...","In a world replete with greed, betrayal, sexua...",Lorenzo Richelmy,Benedict Wong,Joan Chen,Daniel Minahan,David Petrarca,Alik Sakharov,...,Akhat Ibraev,Guy Tannahill,Robyn-Alain Feldman,NaN,NaN,Sean Guest,Alexandra Canosa,Russ Hammonds,2017-10-04,2017-10-11
4,Bloodline,English,A family finds themselves forced to face their...,Intriguing story of a well off family in the F...,Kyle Chandler,Linda Cardellini,Norbert Leo Butz,Mikael Håfström,Todd A. Kessler,Ed Bianchi,...,NaN,NaN,Greer Yeaton,Jeff Shakoor,NaN,Deen Dioria,Michael Carroll,NaN,2017-10-04,2017-10-11
5,Daredevil,English,"A blind lawyer by day, vigilante by night. Mat...",As a child Matt Murdock was blinded by a chemi...,Charlie Cox,Deborah Ann Woll,Elden Henson,Phil Abraham,Stephen Surjik,Peter Hoar,...,NaN,NaN,Alex Shevchenko,Emma Fleischer,Daniel Hank,NaN,NaN,NaN,2017-10-04,2017-10-11
6,Sense8,English,A group of people around the world are suddenl...,Sense8 tells the story of eight strangers: Wil...,Doona Bae,Jamie Clayton,Tina Desai,Lana Wachowski,Lilly Wachowski,James McTeigue,...,Pravesh Sahni,Leifur B. Dagfinnsson,Sheyna Kathleen Smith,Tabrez Noorani,NaN,Aimee Allegretti,Matt Bilski,Loranne Turgeon,2017-10-04,2017-10-11
7,Narcos,English\n|\nSpanish,A chronicled look at the criminal exploits of ...,Narcos tells the true-life story of the growth...,Pedro Pascal,Wagner Moura,Boyd Holbrook,Andres Baiz,Josef Kubota Wladyka,Fernando Coimbra,...,Mariano Carranco,NaN,Jesse Rose Moore,Batan Silva,Matthew Colonna,David Straiton,NaN,NaN,2015-08-28,2015-09-04
8,Jessica Jones,English,Following the tragic end of her brief superher...,Ever since her short-lived stint as a superher...,Krysten Ritter,Rachael Taylor,Eka Darville,Stephen Surjik,Jennifer Getzinger,Uta Briesewitz,...,NaN,NaN,Matt McInnis,Micah Schraft,Hans van Doornewaard,Matthew McCluggage,Kiegan Downs,NaN,2017-10-04,2017-10-11
9,The Get Down,English,A ragtag group of teenagers run wild in the st...,The Get Down focuses on 1970s New York City - ...,Justice Smith,Shameik Moore,Herizen F. Guardiola,Ed Bianchi,Mack Williams,Andrew Bernstein,...,NaN,NaN,Jamie Crowell,Sam Bromell,Zee Hatley,Anthony Talauega,Richmond Talauega,Ed Mundy,2017-10-04,2017-10-11


In [18]:
ua = UserAgent()

search_term = "House of Cards"
google_url = "https://trends.google.com/trends/explore?geo=IN&q="+search_term+",Antarctica"

response = requests.get(google_url) #{"User-Agent": ua.random})
soup = BeautifulSoup(response.text, "html.parser")

In [19]:
soup.text

"Error 429 (Too Many Requests)!!1429. That’s an error.We're sorry, but you have sent too many requests to us recently. Please try again later. That’s all we know."

In [9]:
print(fake_useragent.VERSION)

0.1.11


In [11]:
print(dir(fake_useragent))

['FakeUserAgent', 'FakeUserAgentError', 'UserAgent', 'UserAgentError', 'VERSION', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', 'absolute_import', 'errors', 'fake', 'log', 'settings', 'unicode_literals', 'utils']


In [14]:
ua.random

'Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/537.36 (KHTML like Gecko) Chrome/44.0.2403.155 Safari/537.36'

In [15]:
ua.random

'Mozilla/5.0 (Windows; U; Windows NT 6.1; de-DE) AppleWebKit/533.20.25 (KHTML, like Gecko) Version/5.0.3 Safari/533.19.4'

In [21]:
import sys
!{sys.executable} -m pip install pytrends

In [22]:
!{sys.executable} -m pip install proxy-requests

  Created wheel for proxy-requests: filename=proxy_requests-0.5.1-py3-none-any.whl size=4139 sha256=ea9cc58c38dbb0b04781001222e1dbd44746e12ed2d47cec9c581bf648d7df40
  Stored in directory: c:\users\apoor\appdata\local\pip\cache\wheels\03\ac\78\9cfa0be7c25e28024e2da28f4bd6983148c56936b721134253
Successfully built proxy-requests


In [23]:
from proxy_requests import ProxyRequests
r = ProxyRequests(google_url)
r.get()

In [27]:
print(r)

<!DOCTYPE html><html lang=en><meta charset=utf-8><meta name=viewport content="initial-scale=1, minimum-scale=1, width=device-width"><title>Error 429 (Too Many Requests)!!1</title><style nonce="qYO9BrZcvoVOsaCnYCp4Vg">*{margin:0;padding:0}html,code{font:15px/22px arial,sans-serif}html{background:#fff;color:#222;padding:15px}body{color:#222;text-align:unset;margin:7% auto 0;max-width:390px;min-height:180px;padding:30px 0 15px;}* > body{background:url(//www.google.com/images/errors/robot.png) 100% 5px no-repeat;padding-right:205px}p{margin:11px 0 22px;overflow:hidden}pre{white-space:pre-wrap;}ins{color:#777;text-decoration:none}a img{border:0}@media screen and (max-width:772px){body{background:none;margin-top:0;max-width:none;padding-right:0}}#logo{background:url(//www.google.com/images/branding/googlelogo/1x/googlelogo_color_150x54dp.png) no-repeat;margin-left:-5px}@media only screen and (min-resolution:192dpi){#logo{background:url(//www.google.com/images/branding/googlelogo/2x/googlelog

In [4]:
# Start running from here for Google Trends!
country = "US"
date1 = "2014-02-20"
date2 = "2014-02-27"
term = "House of Cards"
url = "https://trends.google.com/trends/explore?date="+date1+"%20"+date2+"&geo="+country+"&q="+term+",Antarctica"
url

'https://trends.google.com/trends/explore?date=2014-02-20%202014-02-27&geo=US&q=House of Cards,Antarctica'

In [5]:
df = pd.read_excel("input-refined.xlsx")
country = "US"

NameError: name 'pd' is not defined

In [12]:
df[["name", "release date", "seven days after"]].head(10)

,name,release date,seven days after
0,House of Cards,2014-02-20,2014-02-27
1,Hemlock Grove,2016-01-04,2016-01-11
2,Orange Is the New Black,2017-10-04,2017-10-11
3,Marco Polo,2017-10-04,2017-10-11
4,Bloodline,2017-10-04,2017-10-11
5,Daredevil,2017-10-04,2017-10-11
6,Sense8,2017-10-04,2017-10-11
7,Narcos,2015-08-28,2015-09-04
8,Jessica Jones,2017-10-04,2017-10-11
9,The Get Down,2017-10-04,2017-10-11


In [15]:
url = "https://trends.google.co.uk/trends/explore?date=2014-02-20%202014-02-27&geo=US&q=House of Cards,Antarctica"
driver.get(url)

In [1]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import os
import time

In [4]:
def download_csv(url, term):
    driver.get(url)

    download_xpath = "(//button[@class='widget-actions-item export'])[1]"
    time.sleep(10)
    elem = element_wait(driver, download_xpath)
    elem.click()
    
    src = "C:\\Users\\apoor\\Downloads\\multiTimeline.csv"
    dest = "./us-files/"+term+".csv"

    while not os.path.exists(src):
        time.sleep(1)

    os.rename(src, dest)
    

In [3]:
def element_wait(driver, xpath):
    wait = WebDriverWait(driver, 10)
    elem = wait.until(EC.element_to_be_clickable((By.XPATH, xpath)))
    return elem

In [2]:
driver = webdriver.Chrome('./driver/chromedriver')
# driver = webdriver.Edge('./edgedriver/msedgedriver.exe')
# driver.quit()

In [54]:
country = "US"
for i in df["name"][317:].index:
    date1 = df["release date"][i]
    date2= df["seven days after"][i]
    term = df["name"][i].strip()
    newterm = term.replace(" ", "%20")
    newterm = term.replace(",", "")
    newterm = term.replace(".", "")
    newterm = term.replace("/", "")
    newterm = term.replace("\\", "")
    newterm = term.replace(":", "")
    newterm = term.replace("#", "")
    newterm = term.replace("!", "")
#     print(term)
#     print(date1)
#     print(date2)
    if str(date1) != "nan":
        print(term)
#         continue
    url = "https://trends.google.co.uk/trends/explore?date="+date1+"%20"+date2+"&geo="+country+"&q="+newterm+",Antarctica"
#     print(term)
#     print(url)
    download_csv(url, term)
    time.sleep(10)

BlackAF
Richie Rich
Alexa and Katie
Forever
Mirzapur
Breathe Into the Shadows
Inside Edge
Laakhon Mein Ek
Pushpavalli
Shaitaan Haveli
Chacha Vidhayak Hain Humare
Four More Shots Please
Made in Heaven
Mind The Malhotras
The Family Man
Hostel Daze
The Forgotten Army Azaadi ke liye
Afsos
Panchayat
Paatal Lok
Bandish Bandits
GangStars
Going Viral Pvt. Ltd.
Sacred Games
Ghoul
Selection Day
Delhi Crime
Leila
Typewriter
Bard of Blood
Jamtara Sabka Number Ayega
Taj Mahal 1989
Hasmukh
Betaal
Masaba Masaba
Stories by Rabindranath Tagore
Indian Matchmaking
Baahubali The Beginning
Dharmakshetra
Little Things
21 Sarfarosh Saragarhi 1897
Mere Papa, Hero Hiralal
College Romance
Khan No. 1 Crime Hunter
Brij Mohan Amar Rahe
Love Per Square Foot
Lust Stories
Rajma Chawal
Chopsticks
Upstarts
Yeh Ballet
Talvar
Mrs. Serial Killer
Bulbbul
Choked
Gunjan Saxena The Kargil Girl
Raat Akeli Hai
Class of 83
Dolly Kitty and Those Twinkling Stars
Serious Men
Ginny Weds Sunny
Soni
Girls Hostel
Breathe
Rasbhari
She
G

In [53]:
df["name"][i], i

('True and the Rainbow Kingdom', 316)

In [43]:
for i in range(5):
    driver.quit()

In [59]:
import glob
count=0


In [85]:
newdest = "./wrong/"
for file in (glob.glob("./files/*.csv")):
    df=pd.read_csv(file, skiprows=1)
    if "Day" not in (df.columns[0]):
        print(file)
    filename = file.split("\\")
    filename = filename[1].replace(".csv", "")
#     if (filename) not in (df.columns[1]):
#         print(filename)
#         print("./newfiles/"+filename+".csv")
#         print("./wrong/"+filename+".csv")
#         os.rename(file.replace('\\', '/'), newdest+filename+".csv")

./files\Hemlock Grove.csv
./files\Mere Papa, Hero Hiralal.csv


In [ ]:
# ./newfiles\All About The Washingtons.csv
# ./newfiles\Archibald's Next Big Thing.csv
# ./newfiles\Charlie's Colorforms City.csv
# ./newfiles\Dolly Parton's Heartstrings.csv
# ./newfiles\Go! Go! Cory Carson.csv
# ./newfiles\Gortimer Gibbon's Life on Normal Street.csv
# ./newfiles\Harvey Girls Forever!.csv
# ./newfiles\Haters Back Off!.csv
# ./newfiles\It's Bruno!.csv
# ./newfiles\Love, Death and Robots.csv
# ./newfiles\Richie Rich.csv
# ./newfiles\Seven Seconds.csv
# ./newfiles\The Good Cop.csv
# ./newfiles\Trolls The Beat Goes On!.csv
# ./newfiles\We're Lalaloopsy.csv